In [ ]:
%run Archpool.ipynb
%run Argparser.ipynb
%run Topo_treatment.ipynb
%run Net.ipynb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image

import model
from dataset import *
from utils import check_dir
from tqdm import tqdm
import numpy as np
from functools import reduce

import os
import json
import argparse

import sys
import json
sys.path.insert(0, './persis_lib_cpp')
from persis_homo_optimal import *

torch_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def parse_args():
    '''parser = argparse.ArgumentParser()
    parser.add_argument('config', help='configuration file path')
    opt = parser.parse_args()
    with open(opt.config, 'r') as inf:
        config = json.load(inf)'''
    
    f = open('example.json')
    config = json.load(f)

    try:
        if config['output_path'][-1] != '/':
            config['output_path'] += '/'
        if config['train_data_path'][-1] != '/':
            config['train_data_path'] += '/'
        if config['val_data_path'][-1] != '/':
            config['val_data_path'] += '/'
    except KeyError as err:
        print(f'{opt.config}: Unspecified path {err}')
        exit(1)
    return config


def initialize_network(config):
    network = {}
    random_seed = 0
    try:
        config['resume']
        random_seed = config['random_seed']
    except KeyError:
        config['resume'] = False
    torch.manual_seed(random_seed)
    if (config['resume'] and os.path.isfile(config['resume'])):
        confout = config['output_path'] + config['name'] + '/'
        network['resume'] = True
        checkpoint = torch.load(config['resume'], map_location=torch_device)
        network['epoch_start'] = checkpoint['epoch'] + \
            1 if checkpoint['output_dir'] == confout else 0
        network['epoch_end'] = config['epoch'] or checkpoint['epoch_end']
        network['output_dir'] = confout
        network['checkpoint_dir'] = checkpoint['checkpoint_dir']
        network['learning_rate'] = checkpoint['learning_rate']
        network['train_data_dir'] = checkpoint['train_data_dir']
        network['val_data_dir'] = checkpoint['val_data_dir']
        network['name'] = checkpoint['name']
        network['batch_size'] = checkpoint['batch_size']
        network['features'] = checkpoint['features']
        network['image_size'] = checkpoint['image_size']
        network['image_channels'] = checkpoint['image_channels']
        network['optimizer_name'] = checkpoint['optimizer_name']
        network['arch'] = checkpoint['arch']
        network['bn'] = checkpoint['bn']
        network['checkpoint'] = checkpoint
    else:
        network['resume'] = False
        try:
            network['output_dir'] = config['output_path'] + \
                config['name'] + '/'
            network['name'] = config['name']
            network['epoch_start'] = 0
            network['epoch_end'] = config['epoch']
            network['learning_rate'] = config['learning_rate']
            network['batch_size'] = config['batch_size']
            network['features'] = int(config['features'])
            network['image_size'] = config['image_size']
            network['image_channels'] = config['image_channels']
            network['optimizer_name'] = config['optimizer']
            network['train_data_dir'] = config['train_data_path']
            network['val_data_dir'] = config['val_data_path']
            network['arch'] = config['arch']
            network['bn'] = config['bn']
        except KeyError as err:
            print(f'Configuration: Unspecified field {err}')
            exit(1)
    network['checkpoint_dir'] = network['output_dir'] + 'checkpoints/'
    network['result_dir'] = network['output_dir'] + 'result/'
    check_dir(network['output_dir'])
    check_dir(network['checkpoint_dir'])
    check_dir(network['result_dir'])

    network['logfile_path'] = network['result_dir'] + 'logfile.txt'
    network['performance_path'] = network['result_dir'] + 'performance.txt'
    learning_model = model.AutoEncoder(
        network['image_size'], network['image_channels'], network['features'], network['arch'], network['bn'])
    #learning_model.load_state_dict(torch.load("C:/Users/nelsite/Desktop/Coding_with_Fahim/Topological_Segmentation/TopoSegNetSimple/network/Output/Unet_Training/checkpoints/Unet_Training_49.pth"))
    learning_model = learning_model.to(torch_device)
    
    network['loss_function'] = WeightedBCELoss(one_weight=1,zeros_weight=1)
    # network['loss_function'] = nn.MSELoss()
    optimizer = None
    if network['optimizer_name'] == 'adam':
        optimizer = optim.Adam(learning_model.parameters(),
                               lr=network['learning_rate'])
    elif network['optimizer_name'] == 'sgd':
        optimizer = optim.SGD(learning_model.parameters(),
                              momentum=0.9, weight_decay=1e-2,
                              lr=network['learning_rate'])

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=1)

    if (network['resume']):
        learning_model.load_state_dict(network['checkpoint']['model'])
        optimizer.load_state_dict(network['checkpoint']['optimizer'])
        scheduler.load_state_dict(network['checkpoint']['scheduler'])
    
    network['model'] = learning_model
    network['optimizer'] = optimizer
    network['scheduler'] = scheduler
    return network


class WeightedBCELoss:
    def __init__(self, one_weight=1.0, zeros_weight=1.0, reduction="mean"):
        self.reduction = reduction
        self.update_weights(one_weight, zeros_weight)

    def update_weights(self, one_weight, zeros_weight):
        self.weights = torch.FloatTensor([one_weight, zeros_weight])
        self.weights.to(torch_device)

    def _bce(self, x, y):
        weights = -self.weights
        x = torch.clamp(x, min=1e-7, max=1-1e-7)
        y = torch.clamp(y, min=1e-7, max=1-1e-7)
        return weights[1]*y*torch.log(x) + weights[0]*(1-y)*torch.log(1-x)

    def __call__(self, pred, truth):
        loss = self._bce(pred, truth)
        if self.reduction == 'mean':
            return torch.mean(loss)
        if self.reduction == 'sum':
            return torch.sum(loss)
        return loss



def train(adv_params, network, dataloader, withTopo=True):
    
    et = Edges_(adv_params, debug=False)
    #criterionT = GANLoss("vanilla_topo", "sum").to(torch_device)
    criterionT = nn.BCEWithLogitsLoss(reduction="sum")
    loss_function = network['loss_function']
    model = network['model']
    optimizer = network['optimizer']
    model.train()
    running_loss = 0.0
    t_loss = 0.0
    alpha = 0.96
    result_dir = network['result_dir']
    batch_number = 0
    step_num = 0
    for data in dataloader:
        
        step_num = step_num + 1
        print("step_num : ", step_num)
        scalars, label = data

        scalars = scalars.to(torch_device)
        label = label.to(torch_device)
        optimizer.zero_grad()

        p = model(scalars)
        prediction = (p - torch.min(p))/(torch.max(p) - torch.min(p)) *2-1
        
        loss = loss_function(prediction, label)
        running_loss += loss.item()
        print("training loss : ", loss.item())
        
        #loss.backward()
        num_rows = p.size(0)
        s = scalars.cpu().view(
            num_rows, 1, 64, 64).double()
        t = label.cpu().view(
            num_rows, 1, 64, 64).double()

        pred = prediction.cpu().view(
            num_rows, 1, 64, 64).double()
        
        pl = label.cpu().view(
            num_rows, 1, 64, 64).double()
        
        out_image = torch.transpose(torch.stack((s, t, pl,pred)), 0, 1).reshape(
            4*num_rows, 1,  64, 64)
        save_image(out_image.cpu(
        ), f"{result_dir}_batch{batch_number}.png", padding=4, nrow=24)
        batch_number += 1
        
        if withTopo:
            
            tp_wgt   = et.return_tp_weight()
            fake_fix, mean_wasdis = et.fix_with_topo(prediction.detach().cpu().numpy(), label.detach().cpu().numpy(),
                                                     et.return_target_dim(), result_dir, batch_number, num_rows, 
                                                     -1.0, 1.0, blind=et.blind())
            #print( prediction.detach().cpu().numpy().shape, et.return_target_dim())
            fake_fix = torch.from_numpy(fake_fix).to(torch_device)
            
            fake_fix = torch.unsqueeze(fake_fix, 1)
            fake_fix = (fake_fix - torch.min(fake_fix))/(torch.max(fake_fix) - torch.min(fake_fix))
            errT = criterionT(p, fake_fix) * tp_wgt
            t_loss += errT.item()
            print("Topo loss : ", errT.item())
            #errT.backward()
            #total_loss = (alpha * loss) + ((1-alpha) * errT)
            tot_loss = loss + errT
            print("Total loss : ", tot_loss.item())
            tot_loss.backward()
            optimizer.step()
        
        else:
            #loss.backward()
            optimizer.step()
    
    training_loss = running_loss / len(dataloader.dataset)
    topo_loss = t_loss / len(dataloader.dataset)
    total_loss = tot_loss / len(dataloader.dataset)
    
    return [training_loss], [topo_loss], [total_loss]


def validate(network, dataloader, epoch):
    image_size = [network['image_size'], network['image_size']]
    running_loss = 0.0
    tp = 0.0  # true positive
    tn = 0.0  # true negative
    fp = 0.0  # false positive
    fn = 0.0  # false negative

    l1_diff = 0.0
    with torch.no_grad():
        loss_function = network['loss_function']
        model = network['model']
        result_dir = network['result_dir']
        image_channels = network['image_channels']
        model.eval()
        batch_number = 0
        output_image = False
        for i, data in enumerate(dataloader):
            scalars, label = data
            label = label.to(torch_device)
            scalars = scalars.to(torch_device)
            batch_size = label.size(0)

            prediction = model(scalars)
            '''pred = prediction.cpu().detach().numpy()
            pred = np.squeeze(pred)
            fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(12, 4))
            for i in range(5):
                ax[i].imshow(pred[i], cmap='gray')
                ax[i].axis('off')

            plt.show()'''
            
            loss = loss_function(prediction, label)
            running_loss += loss.item() * batch_size
            # log accuracy
            pred = prediction.cpu().view(batch_size, -1).double()
            truth = label.cpu().view(batch_size, -1).double()

            plabel = torch.zeros(pred.size())
            plabel[pred >= 0.3] = 1
            tp += torch.sum(torch.logical_and(plabel == 1, truth == 1).float())
            tn += torch.sum(torch.logical_and(plabel == 0, truth == 0).float())
            fp += torch.sum(torch.logical_and(plabel == 1, truth == 0).float())
            fn += torch.sum(torch.logical_and(plabel == 0, truth == 1).float())

            l1_diff += torch.sum(torch.abs(pred - truth))

            if epoch != "":
                if (epoch == network['epoch_end'] - 1) or (i == len(dataloader) - 1):
                    output_image = True

            #if output_image:
            num_rows = batch_size
            s = scalars.cpu().view(
                num_rows, 1, image_size[1], image_size[0]).double()
            t = label.cpu().view(
                num_rows, 1, image_size[1], image_size[0]).double()

            pred = prediction.cpu().view(
                num_rows, 1, image_size[1], image_size[0]).double()
            
            pl = plabel.cpu().view(
                num_rows, 1, image_size[1], image_size[0]).double()
            
            out_image = torch.transpose(torch.stack((s, t, pl,pred)), 0, 1).reshape(
                4*num_rows, 1,  image_size[1], image_size[0])
            save_image(out_image.cpu(
            ), f"{result_dir}epoch_{epoch}_batch{batch_number}.png", padding=4, nrow=24)
            batch_number += 1
        # end for loop
    # end with nograd
    val_loss = running_loss/len(dataloader.dataset)
    l1_diff /= len(dataloader.dataset)
    tp /= len(dataloader.dataset)
    tn /= len(dataloader.dataset)
    fp /= len(dataloader.dataset)
    fn /= len(dataloader.dataset)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = (tp)/(tp+fp)
    recall = (tp)/(tp+fn)
    f1 = 2*tp / (2 * tp + fp + fn)

    return [val_loss], [accuracy, precision, recall, f1, l1_diff]


def floats2str(l):
    return ",".join(map(lambda x: f'{x:.6f}', l))


def parameters_count(model):
    total = 0
    total_t = 0
    for p in model.parameters():
        if p.requires_grad:
            total += p.numel()
            total_t += p.numel()
        else:
            total += p.numel()
    return total, total_t


def main():
    adv_params  = return_advanced_params()
    config = parse_args()
    network = initialize_network(config)

    p, pt = parameters_count(network['model'])
    print(f'number of parameters(trainable) {p}({pt})')

    with open(network['output_dir']+'config.json', 'w') as jsonout:
        json.dump(config, jsonout, indent=2)

    train_dataset = ImageBoundary(
        config['train_data_path'], network['image_channels'])
    #i,t = train_dataset[0]
    #print(i,t)
    train_dataloader = DataLoader(
        train_dataset, batch_size=network['batch_size'], shuffle=True)
    val_dataset = ImageBoundary(
        config['val_data_path'], network['image_channels'])
    val_dataloader = DataLoader(
        val_dataset, batch_size=network['batch_size'], shuffle=False)

    if network['resume']:
        logfile = open(network['logfile_path'], 'a')
        perf_log = open(network['performance_path'], 'a')
    else:
        logfile = open(network['logfile_path'], 'w')
        logfile.write('epoch,train_loss,val_loss,topo_loss,total_loss\n')
        perf_log = open(network['performance_path'], 'w')
        perf_log.write(
            'epoch, accuracy, precision, recall, f1, l1_diff_per_image)\n')

    for epoch in tqdm(range(network['epoch_start'], network['epoch_end'])):
        t_loss, topo_loss, total_loss = train(adv_params, network, train_dataloader, True)
        v_loss, performance = validate(network, val_dataloader, epoch)
        network['scheduler'].step(total_loss[0])

        performance = floats2str(performance)
        perf_log.write(f'{epoch},{performance}\n')
        perf_log.flush()

        t_loss = floats2str(t_loss)
        v_loss = floats2str(v_loss)
        topo_loss = floats2str(topo_loss)
        total_loss = floats2str(total_loss)
        
        logfile.write(f'{epoch},{t_loss},{v_loss},{topo_loss},{total_loss}\n')
        logfile.flush()
        if ((epoch+1) % 50 == 0) or epoch == network['epoch_end'] - 1:
            torch.save({
                'epoch': epoch,
                'epoch_end': network['epoch_end'],
                'model': network['model'].state_dict(),
                'optimizer': network['optimizer'].state_dict(),
                'optimizer_name': network['optimizer_name'],
                'scheduler': network['scheduler'].state_dict(),
                'checkpoint_dir': network['checkpoint_dir'],
                'train_data_dir': network['train_data_dir'],
                'val_data_dir': network['val_data_dir'],
                'output_dir': network['output_dir'],
                'name': network['name'],
                'batch_size': network['batch_size'],
                'learning_rate': network['learning_rate'],
                'features': network['features'],
                'image_size': network['image_size'],
                'image_channels': network['image_channels'],
                'arch': network['arch'],
                'bn': network['bn']
            }, f'{network["checkpoint_dir"]}{network["name"]}_{epoch}.pth')
    logfile.close()
    perf_log.close()


if __name__ == '__main__':
    main()

number of parameters(trainable) 7762465(7762465)


  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

step_num :  1
training loss :  0.12208235263824463


C:\ProgramData\Anaconda3\envs\topo_gan\lib\site-packages\ipykernel_launcher.py:504: DeprecationWarning: Call to deprecated class PersImage. (Replaced with the class `persim.PersistenceImager`.) -- Deprecated since version 0.1.5.


Computing 1D 1-wasserstein distance.
Topo loss :  2.7073092460632324
Total loss :  2.8293914794921875
step_num :  2
training loss :  0.13000595569610596
Computing 1D 1-wasserstein distance.
Topo loss :  2.696916103363037
Total loss :  2.8269219398498535
step_num :  3
training loss :  0.0883340984582901
Computing 1D 1-wasserstein distance.
Topo loss :  2.7074947357177734
Total loss :  2.7958288192749023
step_num :  4
training loss :  0.07925327122211456
Computing 1D 1-wasserstein distance.
Topo loss :  2.6816442012786865
Total loss :  2.760897397994995
step_num :  5
training loss :  0.09542062878608704
Computing 1D 1-wasserstein distance.
Topo loss :  2.6848058700561523
Total loss :  2.780226469039917
step_num :  6
training loss :  0.11553647369146347
Computing 1D 1-wasserstein distance.
Topo loss :  2.6909804344177246
Total loss :  2.8065168857574463
step_num :  7
training loss :  0.12326169013977051
Computing 1D 1-wasserstein distance.
Topo loss :  2.66579008102417
Total loss :  2.789

...

Computing 1D 1-wasserstein distance.
Topo loss :  2.7394585609436035
Total loss :  3.3381848335266113
step_num :  164
training loss :  0.6134519577026367
Computing 1D 1-wasserstein distance.
Topo loss :  2.7168426513671875
Total loss :  3.330294609069824
step_num :  165
training loss :  0.5541762113571167
Computing 1D 1-wasserstein distance.
Topo loss :  2.7193336486816406
Total loss :  3.273509979248047
step_num :  166
training loss :  0.5337698459625244
Computing 1D 1-wasserstein distance.
Topo loss :  2.6952764987945557
Total loss :  3.22904634475708
step_num :  167
training loss :  0.6152045130729675
Computing 1D 1-wasserstein distance.
Topo loss :  2.7257237434387207
Total loss :  3.340928316116333
step_num :  168
training loss :  0.4521489143371582
Computing 1D 1-wasserstein distance.
Topo loss :  2.713649034500122
Total loss :  3.1657979488372803
step_num :  169
training loss :  0.5460460782051086
Computing 1D 1-wasserstein distance.
Topo loss :  2.7293949127197266
Total loss : 

Computing 1D 1-wasserstein distance.
Topo loss :  2.7051374912261963
Total loss :  3.310486316680908
step_num :  218
training loss :  0.5638881921768188
Computing 1D 1-wasserstein distance.
Topo loss :  2.7268459796905518
Total loss :  3.29073429107666
step_num :  219
training loss :  0.5252925753593445
Computing 1D 1-wasserstein distance.
Topo loss :  2.715317487716675
Total loss :  3.240610122680664
step_num :  220
training loss :  0.5707108378410339
Computing 1D 1-wasserstein distance.
Topo loss :  2.6831095218658447
Total loss :  3.2538204193115234
step_num :  221
training loss :  0.5601865649223328
Computing 1D 1-wasserstein distance.
Topo loss :  2.7091450691223145
Total loss :  3.269331693649292
step_num :  222
training loss :  0.5916801691055298
Computing 1D 1-wasserstein distance.
Topo loss :  2.7038309574127197
Total loss :  3.295511245727539
step_num :  223
training loss :  0.5263158082962036
Computing 1D 1-wasserstein distance.
Topo loss :  2.7153708934783936
Total loss :  

Computing 1D 1-wasserstein distance.
Topo loss :  2.71413254737854
Total loss :  3.2826461791992188
step_num :  272
training loss :  0.5645084381103516
Computing 1D 1-wasserstein distance.
Topo loss :  2.71626877784729
Total loss :  3.2807772159576416
step_num :  273
training loss :  0.5111933946609497
Computing 1D 1-wasserstein distance.
Topo loss :  2.6906466484069824
Total loss :  3.2018399238586426
step_num :  274
training loss :  0.5520491600036621
Computing 1D 1-wasserstein distance.
Topo loss :  2.711848735809326
Total loss :  3.2638978958129883
step_num :  275
training loss :  0.6299830675125122
Computing 1D 1-wasserstein distance.
Topo loss :  2.7051830291748047
Total loss :  3.3351659774780273
step_num :  276
training loss :  0.5521157383918762
Computing 1D 1-wasserstein distance.
Topo loss :  2.7028164863586426
Total loss :  3.254932165145874
step_num :  277
training loss :  0.5745914578437805
Computing 1D 1-wasserstein distance.
Topo loss :  2.734985589981079
Total loss :  

Computing 1D 1-wasserstein distance.
Topo loss :  2.7274107933044434
Total loss :  3.392958164215088
step_num :  326
training loss :  0.596431314945221
Computing 1D 1-wasserstein distance.
Topo loss :  2.7460780143737793
Total loss :  3.3425092697143555
step_num :  327
training loss :  0.6319884657859802
Computing 1D 1-wasserstein distance.
Topo loss :  2.737217426300049
Total loss :  3.369205951690674
step_num :  328
training loss :  0.6171426177024841
Computing 1D 1-wasserstein distance.
Topo loss :  2.7151246070861816
Total loss :  3.3322672843933105
step_num :  329
training loss :  0.5804171562194824
Computing 1D 1-wasserstein distance.
Topo loss :  2.725879669189453
Total loss :  3.3062968254089355
step_num :  330
training loss :  0.603950560092926
Computing 1D 1-wasserstein distance.
Topo loss :  2.7008237838745117
Total loss :  3.304774284362793
step_num :  331
training loss :  0.5846704840660095
Computing 1D 1-wasserstein distance.
Topo loss :  2.7093465328216553
Total loss :  

Computing 1D 1-wasserstein distance.
Topo loss :  2.7000131607055664
Total loss :  3.2706644535064697
step_num :  380
training loss :  0.5417453050613403
Computing 1D 1-wasserstein distance.
Topo loss :  2.704833507537842
Total loss :  3.2465786933898926
step_num :  381
training loss :  0.4790773093700409
Computing 1D 1-wasserstein distance.
Topo loss :  2.697782039642334
Total loss :  3.1768593788146973
step_num :  382
training loss :  0.6223539710044861
Computing 1D 1-wasserstein distance.
Topo loss :  2.711526393890381
Total loss :  3.3338804244995117
step_num :  383
training loss :  0.5613407492637634
Computing 1D 1-wasserstein distance.
Topo loss :  2.7179713249206543
Total loss :  3.2793121337890625
step_num :  384
training loss :  0.515967845916748
Computing 1D 1-wasserstein distance.
Topo loss :  2.718031644821167
Total loss :  3.233999490737915
step_num :  385
training loss :  0.5992744565010071
Computing 1D 1-wasserstein distance.
Topo loss :  2.7042236328125
Total loss :  3.